In [9]:
import numpy as np
import pandas as pd
import altair as alt
import panel as pn
pn.extension('vega')

pn.__version__

'1.4.1'

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [11]:
index_file =pd.ExcelFile('index_file.xlsx') 

In [41]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display

# 定义 category_mapping 和 name 列表
category_mapping = {
    1: ('WORK', 'Participation', 'Segregation and quality of work'),
    2: ('MONEY', 'Financial resources', 'Economic situation'),
    3: ('KNOWLEDGE', 'Attainment and participation', 'Segregation'),
    4: ('TIME', 'Care activities', 'Social activities'),
    5: ('POWER', 'Political', 'Economic', 'Social'),
    6: ('HEALTH', 'Status', 'Behaviour', 'Access')
}
name=['WORK','MONEY','KNOWLEDGE','TIME','POWER','HEALTH']

# 占位符函数，需要替换为实际的数据提取功能
def get_data(country_name, category):
    df_list = []
    for i, sheet_name in enumerate(index_file.sheet_names):
        df = pd.read_excel(index_file, sheet_name=sheet_name)
        df['Time'] = sheet_name[:4]
        category_names = category_mapping[category]
        df = df[df['Country'] == country_name]
        df = df[['Time', 'Country'] + list(category_names)]  # 保留时间、国家和映射的类别名称的列
        df_list.append(df)  # 添加到数据列表中
    return pd.concat(df_list)


# 创建下拉菜单部件
country_dropdown = widgets.Dropdown(options=['BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR',
                                            'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL',
                                            'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE'],
                                    description='Choose Country:', value='FR')

# 显示下拉菜单
display(country_dropdown)

# 创建一个字典用于存储图表对象
charts = {}

# 国家选择回调函数
def on_country_change(change):
    clear_output()  # 清除先前的输出
    display(country_dropdown)  # 重新显示下拉菜单

    country_name = change['new']
    country_category_data = {}
    for i in range(1, 7):
        country_category_data[i] = get_data(country_name, i)
    for i in range(1, 7):
        country_category_data[i] = pd.DataFrame(country_category_data[i])

    # 对每个类别的数据进行处理和可视化
    for i in range(1, 7):
        data = country_category_data[i]
        data = data.melt(id_vars=['Time', 'Country'], var_name='Category', value_name='Index')
        custom_color = {  
        'WORK':'red',
        'MONEY':'red',
        'KNOWLEDGE':'red',
        'TIME':'red',
        'POWER':'red',
        'HEALTH':'red',
        'Participation':'dodgerblue', 
        'Segregation and quality of work': 'dodgerblue',
        'Financial resources':'dodgerblue', 
        'Economic situation':'dodgerblue',
        'Attainment and participation':'dodgerblue',
        'Segregation':'dodgerblue',
        'Care activities':'dodgerblue', 
        'Social activities':'dodgerblue',
        'Political':'dodgerblue', 
        'Economic':'dodgerblue', 
        'Social':'dodgerblue', 
        'Status':'dodgerblue', 
        'Behaviour':'dodgerblue', 
        'Access':'dodgerblue'
    }
        
        line = alt.Chart(data).mark_line().encode(
            x='Time:O',
            y=alt.Y('Index:Q', title='Index', scale=alt.Scale(domain=(50, 100))),
            color=alt.Color('Category:N', scale=alt.Scale(domain=list(custom_color.keys()), range=list(custom_color.values()))),
            tooltip=['Country', 'Index', 'Category']
        )

        points = alt.Chart(data).mark_point().encode(
            x='Time:O',
            y=alt.Y('Index:Q', title='Index', scale=alt.Scale(domain=(50, 100))),
            color=alt.Color('Category:N', scale=alt.Scale(domain=list(custom_color.keys()), range=list(custom_color.values()))),
            tooltip=['Country', 'Index', 'Category']
        )

        chart = alt.layer(line, points).properties(
            title=name[i - 1],
            height=200,
            width=300  # 设置图表宽度为300像素
        ).interactive()

        # 存储图表对象
        charts[i] = chart
    first_row = alt.hconcat(charts[1], charts[2], charts[3])
    second_row = alt.hconcat(charts[4], charts[5], charts[6])

    combined_chart = alt.vconcat(first_row, second_row)

    # 显示更新后的图表
    display(combined_chart)

# 注册回调函数
country_dropdown.observe(on_country_change, names='value')



Dropdown(description='Choose Country:', index=5, options=('BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES'…

alt.VConcatChart(...)